**Create Markers**

In [1]:
import cv2
from cv2 import aruco
import numpy as np

markerImage=np.zeros((600, 600), dtype=np.uint8)
dictionary =aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
aruco.generateImageMarker(dictionary, 23, 600, markerImage, 1)
cv2.imshow("marker23", markerImage)
cv2.waitKey(0)

-1

In [2]:
cv2.imwrite('test.jpg',markerImage)

True

**Detect Markers**

In [5]:
import cv2
from cv2 import aruco
import numpy as np

markerIds = []
markerIds = np.array(markerIds)
markerCorners = []
rejectedCandidates = []


detectorParams =aruco.DetectorParameters()
dictionary = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
det=aruco.ArucoDetector(dictionary, detectorParams)
frame=cv2.imread('test1.PNG')
gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

(corners, ids, rejected)=cv2.aruco.detectMarkers(gray, dictionary, markerCorners, markerIds, detectorParams, rejectedCandidates)
print(corners, ids)
aruco.drawDetectedMarkers(frame, corners, ids)
#detected_markers = aruco_display(corners, ids, rejected, frame)
cv2.imshow("Image", frame)
cv2.waitKey(0)

(array([[[ 59.,  38.],
        [546.,  38.],
        [546., 525.],
        [ 59., 525.]]], dtype=float32),) [[23]]


-1

**Marker detection in Video**

In [6]:
import cv2
from cv2 import aruco
import numpy as np
cap=cv2.VideoCapture(0)

markerIds = []
markerIds = np.array(markerIds)
markerCorners = []
rejectedCandidates = []

detectorParams =aruco.DetectorParameters()
dictionary = aruco.getPredefinedDictionary(aruco.DICT_5X5_100)#DICT_6X6_250
det=aruco.ArucoDetector(dictionary, detectorParams)
while True:
    _,frame=cap.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (corners, ids, rejected)=cv2.aruco.detectMarkers(gray, dictionary, markerCorners, markerIds, detectorParams, rejectedCandidates)
    aruco.drawDetectedMarkers(frame, corners, ids)
    
    cv2.imshow("Image", frame)
    if cv2.waitKey(2) & 0xff==27:
        break
cap.release()
cv2.destroyAllWindows()

**PoseEstimation**

*Function for Euler Angles*

In [1]:
def euler_from_quaternion(x, y, z, w):
  
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)
      
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = math.asin(t2)
      
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)
      
    return roll_x, pitch_y, yaw_z

def angles(marker_ids,tvecs,rvecs):
    for i, marker_id in enumerate(marker_ids):
        transform_translation_x = tvecs[i][0][0]
        transform_translation_y = tvecs[i][0][1]
        transform_translation_z = tvecs[i][0][2]
    
        # Store the rotation information
        rotation_matrix = np.eye(4)
        rotation_matrix[0:3, 0:3] = cv2.Rodrigues(np.array(rvecs[i][0]))[0]
        r = R.from_matrix(rotation_matrix[0:3, 0:3])
        quat = r.as_quat()   
         
        # Quaternion format     
        transform_rotation_x = quat[0] 
        transform_rotation_y = quat[1] 
        transform_rotation_z = quat[2] 
        transform_rotation_w = quat[3] 
         
        # Euler angle format in radians
        roll_x, pitch_y, yaw_z = euler_from_quaternion(transform_rotation_x, transform_rotation_y, transform_rotation_z, transform_rotation_w)
         
        roll_x = math.degrees(roll_x)
        pitch_y = math.degrees(pitch_y)
        yaw_z = math.degrees(yaw_z)
        print("transform_translation_x: {}".format(transform_translation_x))
        print("transform_translation_y: {}".format(transform_translation_y))
        print("transform_translation_z: {}".format(transform_translation_z))
        print("roll_x: {}".format(roll_x))
        print("pitch_y: {}".format(pitch_y))
        print("yaw_z: {}".format(yaw_z))

*To run main snippet*

In [2]:
import cv2
from cv2 import aruco
from scipy.spatial.transform import Rotation as R
import numpy as np
import math
cap=cv2.VideoCapture(0)

camera_matrix=np.matrix([[5.23059088e+04,0.00000000e+00,4.61027617e+02],[0.00000000e+00,2.81349018e+04,2.39792783e+02],[0.00000000e+00,0.00000000e+00,1.00000000e+00]])
dist_coeffs=np.matrix([[5.41114976e+01,-1.12639322e+04,3.13476627e-02,-3.04059145e-01,-2.57512806e+01]])

markerIds = []
markerIds = np.array(markerIds)
markerCorners = []
rejectedCandidates = []

detectorParams =aruco.DetectorParameters()
dictionary = aruco.getPredefinedDictionary(aruco.DICT_5X5_100)#DICT_6X6_250
det=aruco.ArucoDetector(dictionary, detectorParams)
while True:
    _,frame=cap.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (corners, ids, rejected)=cv2.aruco.detectMarkers(gray, dictionary, markerCorners, markerIds, detectorParams, rejectedCandidates)
    
    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, obj_points = aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, dist_coeffs)
        #print(rvecs, tvecs)
        for i in range(ids.size):
            frame = cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], length=0.05 )
            #aruco.drawAxis(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.1)
    #calculateAnglesHERE:
        angles(ids,tvecs,rvecs)
    cv2.imshow("Image", frame)
    if cv2.waitKey(2) & 0xff==27:
        break
cap.release()
cv2.destroyAllWindows()


transform_translation_x: -0.01100317992273511
transform_translation_y: 0.15487855233677825
transform_translation_z: 28.46640833697382
roll_x: 122.74748998777481
pitch_y: -3.4033734049786233
yaw_z: 86.36970364154838
transform_translation_x: -0.008437065137612164
transform_translation_y: 0.1402807397256023
transform_translation_z: 27.863174827805093
roll_x: -122.13299398560801
pitch_y: 3.1919132967432433
yaw_z: 85.23361588172848
transform_translation_x: -0.006577814943616029
transform_translation_y: 0.13199609229673426
transform_translation_z: 28.304130557335046
roll_x: -123.27529405880075
pitch_y: 6.1360869374857145
yaw_z: 85.46725238465346
transform_translation_x: -0.004271467556622789
transform_translation_y: 0.12061901016320785
transform_translation_z: 28.044720479598535
roll_x: -123.15788085048047
pitch_y: 6.005980722741537
yaw_z: 86.45038518555648
transform_translation_x: -0.002509001419179117
transform_translation_y: 0.11139301926097765
transform_translation_z: 27.785898711242236
